In [ ]:
# Ideal Weight Prediction using Neural Network in TensorFlow (with Images)
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from IPython.display import Image, display

# 💡 Overview Image
display(Image(url="https://miro.medium.com/v2/resize:fit:800/format:webp/1*wJt_sMvhbN9_Cef0Dp0UqQ.png", width=500))

# 1. Load Dataset
df = pd.read_csv('ideal_weight_dataset.csv')

# 📊 Show data sample
display(df.head())

# 2. Encode Categorical Variable (Gender)
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])  # Male=1, Female=0

# 3. Feature Matrix and Target Variable
X = df[['Height', 'Age', 'Gender']].values
y = df['Ideal_Weight'].values

# 📈 Data Preprocessing Visualization
display(Image(url="https://raw.githubusercontent.com/OMARSHEHATAAA/Ideal_weight_project/main/preprocessing_example.png", width=500))

# 4. Normalize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 5. Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 🧐 Neural Network Architecture Diagram
display(Image(url="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e4/Artificial_neural_network.svg/1024px-Artificial_neural_network.svg.png", width=600))

# 6. Build Neural Network Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer for regression
])

# 7. Compile the Model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 8. Train the Model
history = model.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=150,
    batch_size=16,
    verbose=1
)

# 9. Evaluate the Model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {mae:.2f} kg")

# 10. Plot Training History
plt.figure(figsize=(8,5))
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Val MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error (kg)')
plt.title('Model Training History')
plt.legend()
plt.grid(True)
plt.show()

# 📉 Training Loss Image Example (Optional Placeholder)
display(Image(url="https://raw.githubusercontent.com/OMARSHEHATAAA/Ideal_weight_project/main/training_curve_example.png", width=500))

# 11. Save Model
model.save('ideal_weight_neural_net_model.h5')
print("Model saved as 'ideal_weight_neural_net_model.h5'")